In [27]:
import os
import cv2
import numpy as np
import torch
from mmocr.apis.inferencers import MMOCRInferencer
from mmocr.utils import poly2bbox

In [28]:
# Cấu hình
input_dir = "./inputs"  # Thư mục chứa ảnh sản phẩm
output_dir = "./output/"  # Thư mục lưu kết quả
os.makedirs(output_dir, exist_ok=True)

In [29]:
# Khởi tạo MMOCR Inferencer
mmocr_inferencer = MMOCRInferencer(
    det="mmocr_dev/configs/textdet/dbnetpp/dbnetpp_swinv2_base_w16_in21k.py",  # Mô hình phát hiện
    det_weights="checkpoints/mmocr/db_swin_mix_pretrain.pth",  # Checkpoint phát hiện
    rec="mmocr_dev/configs/textrecog/abinet/abinet_20e_st-an_mj.py",  # Mô hình nhận dạng
    rec_weights="checkpoints/mmocr/abinet_20e_st-an_mj_20221005_012617-ead8c139.pth",  # Checkpoint nhận dạng
    device="cpu",  # Hoặc "cuda" nếu bạn có GPU
)

Loads checkpoint by local backend from path: checkpoints/mmocr/db_swin_mix_pretrain.pth
11/18 16:01:10 - mmengine - INFO - Delete `relative_position_index` and `relative_coords_table` since we always re-init these params according to the `window_size`, which might cause unwanted but unworried warnings when loading checkpoint.
The model and loaded state dict do not match exactly

missing keys in source state_dict: backbone.stages.0.blocks.0.attn.w_msa.relative_coords_table, backbone.stages.0.blocks.0.attn.w_msa.relative_position_index, backbone.stages.0.blocks.1.attn.w_msa.relative_coords_table, backbone.stages.0.blocks.1.attn.w_msa.relative_position_index, backbone.stages.1.blocks.0.attn.w_msa.relative_coords_table, backbone.stages.1.blocks.0.attn.w_msa.relative_position_index, backbone.stages.1.blocks.1.attn.w_msa.relative_coords_table, backbone.stages.1.blocks.1.attn.w_msa.relative_position_index, backbone.stages.2.blocks.0.attn.w_msa.relative_coords_table, backbone.stages.2.blocks.0

In [30]:
# Hàm trích xuất văn bản từ ảnh và lưu vào thư mục đầu ra
def extract_text(image_path, output_dir, mmocr_inferencer):
    # Chạy inference và lưu ảnh đã xử lý vào output_dir
    print(f"Processing: {image_path}")
    results = mmocr_inferencer(image_path, save_vis=True, out_dir=output_dir)

    # Lấy kết quả văn bản từ mô hình
    texts = []
    for prediction in results["predictions"]:
        for text in prediction["rec_texts"]:
            texts.append(text)

    # Tạo tên file .txt dựa trên tên ảnh gốc
    output_text_file = os.path.join(
        output_dir,
        os.path.basename(image_path).replace(".jpg", ".txt").replace(".png", ".txt"),
    )

    # Lưu file văn bản vào thư mục đầu ra
    with open(output_text_file, "w", encoding="utf-8") as f:
        f.write("\n".join(texts))

    print(f"Extracted texts saved to: {output_text_file}")
    return texts


# Chạy trên tất cả ảnh trong thư mục đầu vào và lưu vào thư mục đầu ra
image_files = [
    os.path.join(input_dir, f)
    for f in os.listdir(input_dir)
    if f.endswith((".jpg", ".png"))
]

for image_file in image_files:
    extract_text(image_file, output_dir, mmocr_inferencer)

Output()

Processing: ./inputs\0.jpg


Output()

Extracted texts saved to: ./output/0.txt
Processing: ./inputs\erase_1.jpg


Output()

...

Output()

Extracted texts saved to: ./output/erase_4.txt
Processing: ./inputs\erase_vis.png


Output()

Extracted texts saved to: ./output/erase_vis.txt
Processing: ./inputs\ex1.jpg


Extracted texts saved to: ./output/ex1.txt
